# Bitcoin Key Pairs

In [20]:
# pip install ecdsa

## Manual Key Generation

In [21]:
import ecdsa
import binascii
import hashlib


# # Generate a private key
private_key_raw_object = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key raw:", (private_key_raw_object.to_string().hex()) + '\x01')

private_key_raw = private_key_raw_object.to_string()

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_raw, curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key:", private_key.to_string().hex())

# Get the public key
# public_key: ecdsa.VerifyingKey = private_key.get_verifying_key()
public_key = private_key.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key: {public_key.to_string("compressed").hex()}")


Private key raw: f0369563b1ef8a03914b5bfe916428ddc0891a80ee89dacc087fbe28c96d60e9
Private key: f0369563b1ef8a03914b5bfe916428ddc0891a80ee89dacc087fbe28c96d60e9
Public key: 033fce724006fcc3ad6299d70736d22b26e594fc1545f122301670a9cbd87d7ab8


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [22]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string('compressed')).digest()
print("SHA256: ", sha256.hex())

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()
print("RIPEMD-160: ", hashed_public_key.hex())

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]
print("Address checksum: ", address_checksum.hex())

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum
print("Binary Bitcoin Address: ", binary_bitcoin_address.hex())

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode()
print("Bitcoin address: ", bitcoin_address)

SHA256:  d5f381098cec9c41cc30cfe0381cd2198a881f62f61cec3587aced4480e3a759
RIPEMD-160:  629149d9b3356047bafe680c93eeabe212f88bed
Address checksum:  e1acc89a
Binary Bitcoin Address:  00629149d9b3356047bafe680c93eeabe212f88bede1acc89a
Bitcoin address:  19zBGcx3vR3J1JHQBXuWH2geHyq9h13nTK


### Verify a bitcoin address

In [23]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [24]:
def encode_varint(i: int) -> bytes:
    """
    Encode a potentially very large integer into varint bytes. The length should be
    specified in little-endian.

    https://bitcoin.org/en/developer-reference#compactsize-unsigned-integers
    """
    if i < 253:
        return bytes([i])
    elif i < 0x10000:
        return b"\xfd" + i.to_bytes(2, "little")
    elif i < 0x100000000:
        return b"\xfe" + i.to_bytes(4, "little")
    elif i < 0x10000000000000000:
        return b"\xff" + i.to_bytes(8, "little")
    else:
        raise ValueError("Integer is too large: %d" % i)

In [25]:
# Path: concepts_bitcoin_transactions.ipynb
import base64

private_key_raw_1 = private_key_raw_object
msg = b"Hello world!"
msg_size = encode_varint(len(msg))
message_hash = hashlib.sha256(hashlib.sha256(b'\x18Bitcoin Signed Message:\n' + msg_size + msg).digest()).digest()
print("Message hash:", message_hash.hex())

signature_ = private_key_raw_1.sign_deterministic(message_hash)
# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
# message = b"\x18Bitcoin Signed Message:\n" + msg_size + msg
# message_size = encode_varint(len(message))
# signature_ = private_key.sign_deterministic(hashlib.sha256(hashlib.sha256(message).digest()).digest())
# print("Signature:", signature_.hex())
print(f'Signature in b64: {base64.b64encode(signature_).hex()}')
print("Signature hex:", binascii.hexlify(signature_).decode('ascii'))

# Verify the signature
print(public_key.verify(signature_, message_hash))


Message hash: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature in b64: 377930486361515345667378486d386f453168667250396a73376e3765414b516b385a6e3972735662522f656e73692b2b45664c3548676b6257372f6c37464a757774506e49794c44467959653366357777426d45773d3d
Signature hex: ef2d0771a41211fb311e6f2813585facff63b3b9fb78029093c667f6bb156d1fde9ec8bef847cbe478246d6eff97b149bb0b4f9c8c8b0c5c987b77f9c3006613
True


In [26]:
# Get the private key
# private_key_hex = 'your_private_key_here'
# private_key_bytes = bytes.fromhex(private_key_hex)
# private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

# The message to sign
# message = 'your_message_here'

# Sign the message
signature = private_key_raw_1.sign(message_hash)
print(f"Signature: {signature.hex()}")

# Print the signature in base64 format
signature_b64 = base64.b64encode(signature).hex()
print(f"Signature b64: {signature_b64}")

# Get the public key
public_key: ecdsa.VerifyingKey = private_key_raw_1.get_verifying_key()
print(f"Public key: {public_key.to_string('compressed').hex()}")


Signature: 77c23ba57f1decc00b962abb89a83c604f30a0610393773e7ca30ebf5aa42332f79bd96c502c0cf5171a7d4ef03fad125ba81ca938aa82feee04dffce8545544
Signature b64: 6438493770583864374d414c6c69713769616738594538776f4745446b33632b664b4d4f7631716b497a4c336d396c735543774d395263616655377750363053573667637154697167763775424e2f383646525652413d3d
Public key: 033fce724006fcc3ad6299d70736d22b26e594fc1545f122301670a9cbd87d7ab8


### Verify a message

In [27]:
# Use the public key to verify a message
# Verify the signature
try:
    public_key.verify(signature, message_hash)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Signature valid


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [28]:
# pip install pybitcoin

In [29]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [30]:
# pip install bitcoin-utils

In [31]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [32]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [33]:
# pip install python-bitcoinlib

In [34]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.magic} {bitcoin_msg.message} {bitcoin_msg.GetHash().hex()}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.hex()}")

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: Kxf4uYHjzCHBFLUAJtXwFhmayuzKtggTuynApyqfmKdy6cvtBkLQ
Public key: 028e3c3f5d2e4f8cb92f85262c8255e81d022650c3fa2c724fe19abbf908c52a56
Bitcoin address: 1Lp8fdRd8wiiqWmUoj2n1zQwtY9TpZSVck
Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
b"\xbb\xbe\xfb\x93a\xad\xca\xa2T\xf3l@1\x83KMz\xeb\xaa2\xa3%\x10\x9b\r\xa0A\xc0\x0c/\xeaK\x0c\xcb\xbf\xef9D\x81\x1e\x9b\x1c\xc0\xf61\xf6\x86\xb5Q5'@\xf5,s\xbb\xe7\xd0c\xad\xe5\xa2|\xa5" 1
Signature: b'ILu++5NhrcqiVPNsQDGDS01666oyoyUQmw2gQcAML+pLDMu/7zlEgR6bHMD2MfaGtVE1J0D1LHO759BjreWifKU='
Signature valid? True


In [35]:
# verify the handmade message from the top
# print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw + b"\x01")}')
print(f'Private key not encoded: {private_key_raw.hex() + "01"}')
# bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw)
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_raw.hex() + "01"), 0)
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
# print(f'Public key base58 Encoded: {base58.b58encode(private_key.get_verifying_key().to_string('compressed'))}')
print(f'bitcoin public not encoded: {private_key.get_verifying_key().to_string('compressed').hex()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
bitcoin_public_key_bytes = private_key.get_verifying_key().to_string('compressed')
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(bitcoin_public_key_bytes.hex()))
print(f"Bitcoin address from pubkey: {bitcoin_address_bytes}")

# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(bytes.fromhex(bitcoin_public_key_bytes.hex()), 0)
# print(f"Bitcoin address: {bitcoin_address_bytes}")


Private key not encoded: f0369563b1ef8a03914b5bfe916428ddc0891a80ee89dacc087fbe28c96d60e901
Bitcoin private key: f0369563b1ef8a03914b5bfe916428ddc0891a80ee89dacc087fbe28c96d60e901
bitcoin public not encoded: 033fce724006fcc3ad6299d70736d22b26e594fc1545f122301670a9cbd87d7ab8
Bitcoin public key: 033fce724006fcc3ad6299d70736d22b26e594fc1545f122301670a9cbd87d7ab8
Bitcoin address from top: 19zBGcx3vR3J1JHQBXuWH2geHyq9h13nTK
Bitcoin address from pubkey: 19zBGcx3vR3J1JHQBXuWH2geHyq9h13nTK


In [36]:

bitcoin_message_bytes = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message} {bitcoin_message_bytes.GetHash().hex()}")

print(f'signature from top: {signature_.decode()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(signature_)
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes.decode()}")

# print(f'signature_bytes_hex: {base64.b64encode(signature_bytes).hex()}')
print(f'signature bytes decoded: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_bytes, bitcoin_message_bytes, signature_bytes)}")

Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
signature from top: ILu++5NhrcqiVPNsQDGDS01666oyoyUQmw2gQcAML+pLDMu/7zlEgR6bHMD2MfaGtVE1J0D1LHO759BjreWifKU=
Signature from top b64: b'SUx1Kys1TmhyY3FpVlBOc1FER0RTMDE2NjZveW95VVFtdzJnUWNBTUwrcExETXUvN3psRWdSNmJITUQyTWZhR3RWRTFKMEQxTEhPNzU5QmpyZVdpZktVPQ=='
b"\xbcD\x89H\xa1c\xee0\x9c\x7f\\\x16;\xedx\x99G\xe7\x80\xbb3a\xcf\x19Uc\xa2\xbf\xd63\r\x85}d\xf6\xa7X\xb8\xad\xe0)\xde\xce'\xf6!\xb8ju\xfd\xb3\xd0\xde\t\x85\x8c\xc1\x83k\x03Q\xa1`\x96" 1
Signature bytes: ILxEiUihY+4wnH9cFjvteJlH54C7M2HPGVVjor/WMw2FfWT2p1i4reAp3s4n9iG4anX9s9DeCYWMwYNrA1GhYJY=
signature bytes decoded: 20bc448948a163ee309c7f5c163bed789947e780bb3361cf195563a2bfd6330d857d64f6a758b8ade029dece27f621b86a75fdb3d0de09858cc1836b0351a16096
Signature valid? True


In [37]:
# Get the private key
private_key_hex = private_key_raw.hex()
private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
print(f"Private key: {private_key.to_string().hex()}")

# Get the public key
public_key_hex = private_key.get_verifying_key().to_string('compressed').hex()
public_key_bytes = binascii.unhexlify(public_key_hex)
public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)
print(f"Public key: {public_key.to_string('compressed').hex()}")

# Get the bitcoin address
bitcoin_address = P2PKHBitcoinAddress.from_pubkey(public_key_bytes)
print(f"Bitcoin address: {bitcoin_address}")

# The message to sign
message = message_hash
print(f"Message: {message.hex()}")

# Sign the message
signature = private_key.sign(message)

# Print the signature in hexadecimal format
print(f"Signature: {binascii.hexlify(signature).decode()}")
print(f"Signature ascii: {binascii.hexlify(signature).decode('ascii')}")

signature_hex = binascii.hexlify(signature).decode('ascii')

# The signature
signature_unhex = binascii.unhexlify(signature_hex.encode('ascii'))
print(f"Signature unhex: {signature_unhex}")

# signature base64
signature_b64 = base64.b64encode(signature_unhex).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Verify the signature
try:
    public_key.verify(signature_unhex, message)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Private key: f0369563b1ef8a03914b5bfe916428ddc0891a80ee89dacc087fbe28c96d60e9
Public key: 033fce724006fcc3ad6299d70736d22b26e594fc1545f122301670a9cbd87d7ab8
Bitcoin address: 19zBGcx3vR3J1JHQBXuWH2geHyq9h13nTK
Message: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: 57d4b36785486d5d4020a76a25a1e45cf20d278f07b10bf4d0e8b4080e4e9b2474be8b6812ef7166f8ddfc92d78a06a0fdf82f9e92c58af7028ebfd9a25f10bc
Signature ascii: 57d4b36785486d5d4020a76a25a1e45cf20d278f07b10bf4d0e8b4080e4e9b2474be8b6812ef7166f8ddfc92d78a06a0fdf82f9e92c58af7028ebfd9a25f10bc
Signature unhex: b"W\xd4\xb3g\x85Hm]@ \xa7j%\xa1\xe4\\\xf2\r'\x8f\x07\xb1\x0b\xf4\xd0\xe8\xb4\x08\x0eN\x9b$t\xbe\x8bh\x12\xefqf\xf8\xdd\xfc\x92\xd7\x8a\x06\xa0\xfd\xf8/\x9e\x92\xc5\x8a\xf7\x02\x8e\xbf\xd9\xa2_\x10\xbc"
Signature b64: V9SzZ4VIbV1AIKdqJaHkXPINJ48HsQv00Oi0CA5OmyR0votoEu9xZvjd/JLXigag/fgvnpLFivcCjr/Zol8QvA==
Signature valid
